In [3]:
import pandas as pd
import numpy as np
import glob
import matplotlib as mpl
import matplotlib.pyplot as plt 
import os

#import more_itertools as mit

In [4]:

FilePathList = "/Users/homefolder/Documents/RapfLab/code/MStest" #sets filepath for initial file/mass list. May not be necessary to keep this way
                                                            #but sets path for list 
FileType="csv" #sets filetype
MassNumbersList=[] #creates an empty list which will be filled by the numbers in the mass list file
os. chdir(FilePathList) #sets working directory to FilePathList
CurrFile = pd.read_csv("NewIncompMassList.csv") #reads the file masslist.csv from the filepath --> since we only want to read one file was easier to just specify file
#print(CurrFile)
MassNumberNames=[]
#######################################


for i in (range(len(CurrFile))):
    MassNumbersList.append(CurrFile['mz_list'][i]) #appends value number i from masslist.csv in the colum mz_list
    MassNumberNames.append(CurrFile['Species'][i])
print(len(MassNumberNames))
print(len(MassNumbersList))

47
47


In [6]:
FilePathList = "/Users/homefolder/Documents/RapfLab/code/MStest/Processed"
FileType="csv" #sets filetype
os. chdir(FilePathList)
Blanks=[]
CurrFile = pd.read_csv("Processedstdev_blank.csv")
for i in (range(len(CurrFile))):
    Blanks.append(CurrFile['standard deviation'][i])

# print(Blanks)
print(len(Blanks))

FileNotFoundError: [Errno 2] No such file or directory: 'Processedstdev_blank.csv'

In [4]:

FilePath = "/Users/timmo/Documents/RapfLab/code/blanktest/files" #Second file path for where the actual files are located
os. chdir(FilePath) #updates file path to the folder with actual files for processing
AllFiles=pd.DataFrame()
FileType="csv" #what filetype are the files. Will probably be csv
files = sorted(glob.glob("*"+FileType)) #gets all files of chosen filetype
num_files = len(files) #how many files are you reading in
OutFolder = "Processed" #folder where processed files will be saved
outname = "" #part of file name. Will be used later
##############################

ErrorRange = 0.00005 #error range. Will be +- to get range and see if values fit in range. 25 ppm error range     
mzlistRep=[]



for entry in range(num_files): #goes through all files
    name = files[entry] 
    CurrFile = pd.read_csv(files[entry],skiprows=1) #reads files
    FullSplit = name.split(".") #gets all parts of the file name
    NameList=(str(FullSplit[0]))
    mzlistRep=[]
    ProcessedX=[] #values for X(Thompsons) will be saved here
    ProcessedY=[] #values for Y(Counts) will be saved here
    PPM=[] #PPM values will be saved here
    names=[]
    BlankRep=[]
    Detected=[]
    #gets the third part of the name (0,1,[2]). Adds it to the name list
    for mass in range(len(MassNumbersList)): #goes through each mast in the MassesList
        MassHigh=((MassNumbersList[mass])+(MassNumbersList[mass]*ErrorRange)) #upper range of Mass
        MassLow=((MassNumbersList[mass])-(MassNumbersList[mass]*ErrorRange)) #lower range of Mass
        fixed=CurrFile.drop(columns="#Point") #drops point column
        fit=fixed["X(Thompsons)"].between(MassLow, MassHigh, inclusive = True) #checks if value for X(Thompson) fits between mass low and mass high
        ####Part two of function
        for i in range(len(fit)): #checks if values in fit are true or false
             if fit[i]== True: #if True then appends values
                    ProcessedX.append(fixed["X(Thompsons)"][i])
                    ProcessedY.append(fixed["Y(Counts)"][i])
                    PPM.append(((abs((fixed["X(Thompsons)"][i])-MassNumbersList[mass]))/MassNumbersList[mass])*1000000)
                    mzlistRep.append(MassNumbersList[mass])
                    names.append(MassNumberNames[mass])
                    BlankRep.append(Blanks[mass])
#                     if fixed(["Y(Counts)"][i])>=(3*Blanks[mass]):
#                     if (fixed["Y(Counts)"][i])>=(3*Blanks[i]):
#                         Detected.append("Not Detected")
#                     else:
#                         Detected.append("Detected")
#                     if ProcessedY[i]>=BlankRep[i]:
#                         print("hi")
                  
            
        if sum(fit) == False: #if all values are false and nothing fits then will append a 0. This makes sure you get the correct
                                  # number of values no matter what
                ProcessedX.append(0)  
                ProcessedY.append(0) #appends 0 for both 
                PPM.append(0)
                mzlistRep.append(MassNumbersList[mass])
                names.append(MassNumberNames[mass])  
                BlankRep.append(Blanks[mass])
#         for i in range(len(ProcessedY)):
#             if ProcessedY[i]>=BlankRep[i]:
#                          print("hi")
#         print(len(ProcessedY))
    for i in range(len(ProcessedY)):
        if ProcessedY[i]>=(3*(BlankRep[i])):
              Detected.append("Detected")
        elif ProcessedY[i]<(3*(BlankRep[i])):
               Detected.append("Not Detected")
        else:
            continue

 
    IndFile = pd.DataFrame(columns = ["Compound Names","mz_list",NameList+"_mass",NameList+"_count",NameList+"_PPM", "Not Detected"]) #columns mz_list, mass, and count +NameList[i]
    IndFile["Compound Names"]=names
    IndFile["mz_list"]=mzlistRep
    IndFile[NameList+"_mass"]=ProcessedX
    IndFile[NameList+"_count"]=ProcessedY #for this and the line before, it reads the values of processed from i*length
    IndFile[NameList+"_PPM"]=PPM
    IndFile["Blank STDev"]=BlankRep
    IndFile["Not Detected"]=Detected
    IndFile.to_csv("/Users/timmo/Documents/RAPFLAB/code/Processed/"+NameList+"_Processed.csv")          #chooses folder
        ####################

#     RowNum=("H"+str(len(ProcessedY)+1)) 
#     print(RowNum)   
#     filename=NameList+"_Processed.csv"
#     workbook  = xlsxwriter.Workbook(filename)
#     worksheet1 = workbook.add_worksheet()
#     format1 = workbook.add_format({'bg_color':   '#FFC7CE',
#                                'font_color': '#9C0006'})
#     worksheet1.conditional_format('A2:'+RowNum, {'type':     'text',
#                                         'criteria': 'begins with',
#                                         'value':    "Detected",
#                                         'format':   format1})
#     workbook.close()

    

ParserError: Error tokenizing data. C error: Expected 3 fields in line 5, saw 4
